In [ ]:
!pip install --upgrade transformers

In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
token = ""

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    token=token
)

In [ ]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [ ]:
import os
import json
from pprint import pprint

def compare(gt, pred):
    messages = [
        {"role": "system", "content": "You will be given a ground truth answer and a model answer. Please output ACCURATE if the model answer matches the ground truth answer or INACCURATE otherwise. Please only return ACCURATE or INACCURATE. It is very important for my job that you do this."},
        {"role": "user", "content": f"""<GroundTruthAnswer>
{gt}
</GroundTruthAnswer>

<ModelAnswer>
{pred}
</ModelAnswer>"""},
    ]
    
#     pprint(messages)

    outputs = pipeline(
        messages,
        max_new_tokens=10,
    )
    return outputs[0]["generated_text"][-1]['content'].lower()

In [ ]:
import os
import json

fpath = "/kaggle/input/gemma-responses/gemma2_response.jsonl"
outpath = "/kaggle/working/gemma2_response_rated.jsonl"

with open(outpath, 'w+') as ff:
    with open(fpath, 'r') as f:
        data = list(f)
        for idx, el in enumerate(data):
            item = json.loads(el)
            gt = item['groundtruth'].replace('<end_of_turn>', '')
            pred = item['prediction'].replace('<end_of_turn><eos>', '')
            response = compare(gt, pred)
            print(idx)
            print(response)
            ff.write(json.dumps({"index": item['index'], "accurate": response}) + '\n')
#             if idx == 100:
#                 break

In [ ]:
fpath = "/kaggle/input/gemma-responses/gemma2_finetuned_response.jsonl"
outpath = "/kaggle/working/gemma2_finetuned_response_rated.jsonl"

with open(outpath, 'w+') as ff:
    with open(fpath, 'r') as f:
        data = list(f)
        for idx, el in enumerate(data):
            item = json.loads(el)
            gt = item['groundtruth'].replace('<end_of_turn>', '')
            pred = item['prediction'].replace('<end_of_turn><eos>', '')
            response = compare(gt, pred)
            print(idx)
            print(response)
            ff.write(json.dumps({"index": item['index'], "accurate": response}) + '\n')
#             if idx == 100:
#                 break